# Starting with Tensorflow

![img](https://www.gstatic.com/devrel-devsite/prod/v870be6fb6841f3532cd3aec5bc0b3146031642f2794ae8ba7f51ebf843a655f9/tensorflow/images/lockup.svg)

[TensorFlow](https://www.tensorflow.org/) and [PyTrorch](http://pytorch.org/) are two of the most commonly used deep learning frameworks. Both of these tools are notable for their ability to compute gradients automatically and do operations on GPU, which can be by orders of magnitude faster than running on CPU. Both libraries serve the same purpose, choosing between them is a matter of preference.

In this tutorial, we'll use Tensorflow for our practical examples.

We'll start from using the low-level core of Tensorflow, and then try out some high-level features.

In [ ]:
import numpy as np
import tensorflow as tf
print(tf.__version__)

In [ ]:
# numpy world

x = np.arange(16).reshape(4, 4)

print(f"X :\n{             x}")
print(f"X.shape : {        x.shape}")
print(f"add 5 :\n{         x + 5}")
print(f"X*X^T :\n{         x @ x.T}")
print(f"mean over rows :\n{x.mean(axis=-1)}")
print(f"cumsum of cols :\n{x.cumsum(axis=0)}")

In [ ]:
# tensorflow world

x = np.arange(16).reshape(4, 4)

x = tf.constant(x, dtype=tf.float32) 

print(f"X :\n{             x}")
print(f"X.shape : {        x.shape}")
print(f"add 5 :\n{         x + 5}")
print(f"X*X^T :\n{         x @ tf.transpose(x)}")
print(f"mean over rows :\n{tf.math.reduce_mean(x, axis=-1)}")
print(f"cumsum of cols :\n{tf.math.cumsum(x, axis=0)}")

## NumPy and Tensorflow

As you can notice, tensorflow allows you to hack achieve the same numpy functionality, be it somewhat different in terms of syntax. There is an experimental feature offering exact numpy interoperability [to some extent](https://www.tensorflow.org/guide/tf_numpy#tensorflow_interoperability), but for now we'll stick with what the core `tensorflow` and `tf.math` is offering.

Note that you may easily convert tensors back to numpy:

In [ ]:
x = tf.range(5, dtype=tf.float32)
tf.reduce_mean(x).numpy()

Tensor operations are subject to type checking, but conversion is not handled automatically. Try to fix the following cell so it does not trow any `TypeError`

In [ ]:
x = tf.constant([1, 2], dtype=tf.int32)
x /= 2.0
x

## Warmup: trigonometric knotwork

There are some simple mathematical functions with cool plots. For one, consider this:

$$ x(t) = t - 1.5 * cos( 15 t) $$
$$ y(t) = t - 1.5 * sin( 16 t) $$

You have `tf.math.cos` and `tf.math.sin` at your disposal 😉

In [ ]:
import matplotlib.pyplot as plt

t = tf.linspace(-10, 10, 10000)

# compute x=x(t) and y=y(t) as defined above
# your code here
raise NotImplementedError

plt.plot(x.numpy(), y.numpy())
plt.show()

In [ ]:
# Check if you plotted the intended equation
assert np.isclose(x[42].numpy(), -9.2157)
assert np.isclose(y[1990].numpy(), -4.6997)


--------

## Automatic gradients

Any self-respecting DL framework must do your *backprop* for you. Tensorflow handles this with the `tf.GradientTape` API. This means computing the gradient of a computation with respect to some inputs, usually tf.Variables. TensorFlow "records" relevant operations executed inside the context of a `tf.GradientTape` onto a "tape". TensorFlow then uses that tape to compute the gradients of a "recorded" computation using reverse mode differentiation.

The general pipeline looks like this:
* When creating a tensor, you define it either as `tf.Tensor` if you don't want it to propagate gradients or `tf.Variable` otherwise:
    * __```tf.zeros(5)```__
    * __```tf.Variable(np.arange(5), dtype=tf.float32)```__
* Define some differentiable `y = arbitrary_function(x)` inside the `tf.GradientTape` context
* Gradients are now available through `GradientTape.gradient(target, sources)`

You may read the [official guide](https://www.tensorflow.org/guide/autodiff) for more detailed explanations.

In [ ]:
x = tf.Variable(0.)

with tf.GradientTape() as tape:
    y = tf.math.sin(x)
    
dy_dx = tape.gradient(y, x)
print(f"sin'(0) = {dy_dx.numpy()}")

Your turn:

In [ ]:
def MSELoss(y, yhat):
    """Mean Squared Error"""
    return tf.reduce_mean((y - yhat)**2)

# initialize our target `y` and our "prediction" `yhat`
y = tf.random.normal([100], 0, 1, tf.float32)
yhat = tf.Variable(tf.random.normal([100], 0, 1, tf.float32))

# Calculate the mean squared error
with tf.GradientTape() as tape:
    # your code here
    # loss = 
    raise NotImplementedError

# Calculate the gradient of loss w.r.t yhat
# dy_dx = 
raise NotImplementedError

print("Checking autograd result equals to analytical derivative.")
print("    Correct Result:", np.all(np.isclose(dy_dx.numpy(), 2 * (yhat - y) / y.shape[0])))

### Learning to solve problems with gradients

We can use the gradient of our loss function to find the parameters corresponding to its minima, this is the basis of most Machine Learning algorithms.

Let's try to solve the following simple equation as an example

 $$ x^3 + 2x - 6 = 0 $$

The necessary steps are:
1. Define equation, variables and a *loss function*:

   In our case the loss function will be the Mean Squared Error as defined bellow. This will quantify how far is our prediction from the solution of the equation $$\mathcal{L}{oss} = \text{MSE}(y_\text{pred}, y_\text{true}) = ||y_\text{pred} - y_\text{true}||^2$$ 
   Where:
   - $y_\text{true} = 0$
   - $y_\text{pred} = x_\text{pred}^3 + 2x_\text{pred} - 6$
   - $x_\text{pred}$ is the solution we are checking, starting with $x_\text{pred} = 1$
2. Calculate $y_\text{pred}$ for the current value of $x_\text{pred}$ and then calculate the $\text{MSE}$ loss
3. Calculate the gradient; this is done by ` tf.GradientTape()` automatically $$\nabla\mathcal{L} = \frac{\partial \mathcal{L}{oss}}{\partial x_\text{pred}} = \frac{\partial \mathcal{L}{oss}}{\partial y_\text{pred}} \frac{\partial y_\text{pred}}{\partial x_\text{pred}}$$
4. Use $\nabla\mathcal{L}$ to find a better $x_\text{pred}$. This step usually involves a scalar hyperparameter called the *learning rate* and denoted $\eta$. 
   $$ x_\text{pred}^{(i+1)} = x_\text{pred}^{(i)} - \eta \nabla\mathcal{L}^{(i)}$$
5. We repeat this $n=$ `N_STEPS` times, iterating over the gradient-based optimization process, saving the $\text{MSE}$ and $x_\text{pred}$ after each step. Thus $i=\overline{1,n}$
   
___
#### Additional Notes: 
For some initial values of $x_\text{pred}$ the optimization process might diverge (like $x_\text{pred}=5$). Since the change in $x_\text{pred}$ is proportional to the gradient of the loss function, if the absolute value of this gradient is very large, $|x_\text{pred}|$ will get larger as well, diverging towards infinity with each step. This issue is called **exploding gradient**. You may avoid this by making the $\eta$ parameter smaller, but this will require more steps to converge.

After running the cell once, experiment with different `lr` and `N_STEPS` in order to converge to the solution when starting from $x_\text{pred}=5$.

In [ ]:
def equation(x):
    return x**3 + 2*x - 6

def MSELoss(y_pred, y_true):
    """Mean Squared Error"""
    return (y_pred- y_true)**2

# Define our variables
x = tf.Variable(2.0)
y_true = tf.constant(0.)

# Optimization hyperparameters
lr = 0.005
N_STEPS = 20

# Lists for intermediate result
losses = []
partial_res = []

for _ in range(N_STEPS):
    with tf.GradientTape() as tape:
        # reinitialize computational graph
        tape.watch(x) # necessary when looping

        # compute prediction and loss value
        y_pred = equation(x)
        loss = MSELoss(y_pred, y_true)
    
    # save loss and intermediary `x`
    losses.append(loss.numpy())
    partial_res.append(x.numpy())

    # get gradient and update `x`
    grad = tape.gradient(loss, x)
    x = x - lr*grad


steps = np.arange(N_STEPS)
error = np.array(partial_res) - 1.45616424613591
plt.figure(figsize=(12,4))
plt.tight_layout()
plt.subplot(1,2,1)
plt.plot(steps, losses)
plt.yscale("log")
plt.xlabel("Step")
plt.ylabel("Loss")
plt.subplot(1,2,2)
plt.plot(steps, error)
plt.xlabel("Step")
plt.ylabel("$x_{pred}$ Absolute Error")
plt.show()

## Homework

Fit a linear model on the following dataset. The linear model definition and loss function are already provided. Your task is to find `a` and `b` which minimizes the loss function on the training set.

In [ ]:
def linear_model(x, a, b):
    return a*x + b 

def reduce_MSE (y_pred, y_true):
    return tf.reduce_mean((y_pred- y_true)**2)

In [ ]:
x_train = np.arange(30)
y_train = 0.2*np.arange(30) + np.random.normal(0, 0.75, 30) - 1

plt.scatter(x_train, y_train);

Implement the linear model fit here:

In [ ]:
# Define our variables
a = tf.Variable(1.0)
b = tf.Variable(0.0)

# Optimization hyperparameters
# lr = 
# N_STEPS = 
raise NotImplementedError

# Lists for intermediate result
losses = []
interm_a = []
interm_b = []

for _ in range(N_STEPS):
    # Use persistent to allow getting multiple gradiens
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(a)
        tape.watch(b)
        raise NotImplementedError

    # save loss and intermediary `a` and `b`
    losses.append(loss.numpy())
    interm_b.append(b.numpy())
    interm_a.append(a.numpy())

    # get gradients and update `a` and `b`
    raise NotImplementedError

###  Visualize fit

In [ ]:
plt.scatter(x_train, y_train);
plt.plot(x_train, linear_model(x_train, interm_a[-1], interm_b[-1]));